In [1]:
import requests
import scrapy

import re
import json

In [2]:
base_url = "http://goheels.com"
url = base_url + "/roster.aspx?path=wvball"
headers = {'User-Agent': 'UNC MEJO Class'}
resp = requests.get(url, headers=headers,)

In [3]:
sel = scrapy.Selector(text=resp.content.decode('utf-8'))

In [4]:
table = sel.css('.sidearm-table')[0]

In [5]:
cols = table.css('tr')[0].css('th').xpath('string()').extract() 
cols

['#',
 'Full Name',
 'Pos.',
 'Ht.',
 'Year',
 'Hometown',
 'High School',
 'Previous School']

In [6]:
other_rows = table.css('tr')[1:]
other_rows

[<Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n    

In [7]:
players = [] #list
for r in other_rows: #for every row
    data = {} #dictionary
    for i, d in enumerate(r.css('td')): #for every cell
        print(i)
        print(d)
        a = d.css('a')
        if a: #if a is not undefined/null
            t = a.xpath('text()').extract()[0]
            data['href'] = a.xpath('@href').extract()[0]
        else:
            try:
                t = d.xpath('text()').extract()[0]
            except:
                t = ""
        data[cols[i]] = t
    players.append(data)
players

0
<Selector xpath='descendant-or-self::td' data='<td class="roster_jerseynum ">1</td>'>
1
<Selector xpath='descendant-or-self::td' data='<td class="sidearm-table-player-name " d'>
2
<Selector xpath='descendant-or-self::td' data='<td class="rp_position_short ">S</td>'>
3
<Selector xpath='descendant-or-self::td' data='<td class="height ">5-10</td>'>
4
<Selector xpath='descendant-or-self::td' data='<td class="roster_class ">So.</td>'>
5
<Selector xpath='descendant-or-self::td' data='<td class="player_hometown ">Colorado Sp'>
6
<Selector xpath='descendant-or-self::td' data='<td class="player_highschool ">Lewis Pal'>
7
<Selector xpath='descendant-or-self::td' data='<td class="player_previous_school "></td'>
0
<Selector xpath='descendant-or-self::td' data='<td class="roster_jerseynum ">2</td>'>
1
<Selector xpath='descendant-or-self::td' data='<td class="sidearm-table-player-name " d'>
2
<Selector xpath='descendant-or-self::td' data='<td class="rp_position_short ">S</td>'>
3
<Selector xpath='

[{'#': '1',
  'Full Name': 'Mariah Evans',
  'High School': 'Lewis Palmer',
  'Hometown': 'Colorado Springs, Colo.',
  'Ht.': '5-10',
  'Pos.': 'S',
  'Previous School': '',
  'Year': 'So.',
  'href': '/roster.aspx?rp_id=8287'},
 {'#': '2',
  'Full Name': 'Kendra Koetter',
  'High School': 'Wesleyan School',
  'Hometown': 'Tampa, Fla.',
  'Ht.': '5-8',
  'Pos.': 'S',
  'Previous School': 'Georgia Southern',
  'Year': 'Jr.',
  'href': '/roster.aspx?rp_id=12641'},
 {'#': '3',
  'Full Name': 'Beth Nordhorn',
  'High School': 'Winter Park',
  'Hometown': 'Winter Park, Fla.',
  'Ht.': '6-7',
  'Pos.': 'MH',
  'Previous School': '',
  'Year': 'Sr.',
  'href': '/roster.aspx?rp_id=8297'},
 {'#': '4',
  'Full Name': 'Taylor Borup',
  'High School': 'Loudoun County',
  'Hometown': 'Leesburg, Va.',
  'Ht.': '6-4',
  'Pos.': 'OH',
  'Previous School': '',
  'Year': 'So.',
  'href': '/roster.aspx?rp_id=8283'},
 {'#': '5',
  'Full Name': 'Taylor Fricano',
  'High School': 'Palatine',
  'Hometown': '

In [8]:
def update_with_bio(player):
    player_url = base_url + player["href"]
    response = requests.get(player_url, headers=headers)
    content = response.content.decode('utf-8')
    sel = scrapy.Selector(text=response.content.decode('utf-8'))
    
    img_div = sel.css('.sidearm-roster-player-image')
    img = img_div.css('img').xpath("@src").extract()[0]
    player["img_url"] = img
    
    bio_div = sel.css('#sidearm-roster-player-bio')
    bio = bio_div.xpath('string()').extract()[0]
    player["bio"] = bio
    
    player["sel"] = sel

In [9]:
js_obj_rx = re.compile(r'.*?responsive-roster-bio\.ashx.*?({.*?})')

In [10]:
def find_stats_url(data):
    clean_objs = []
    for obj_str in data:
        # We only want the stats object...
        if 'stats' not in obj_str:
            continue

        # Remove the braces
        obj_str = obj_str.replace('{', '').replace('}', '')
        # Remove the quotes from whole string since some variables had them and some didn't, now will be able to put
            #proper quotes wherever they need to be in one fell swoop
        obj_str = obj_str.replace("'", '').replace('"', '')
        # Split apart on commas
        obj_pairs = obj_str.split(',')
        # Now it looks like [" type: related", " rp_id: 9450", ...]
        # So we need to wrap everything in quotes to make it json so lets get separate values
        obj_pairs = [x.split(":") for x in obj_pairs]
        # Now it looks like [(' type', ' related'), ]
        # and we need to clean up all the leading / trailing white space
        # and wrap each value in double quotes
        
        clean_pairs = []
        for pair in obj_pairs:
            clean_pairs.append(['"{}"'.format(p.strip()) for p in pair]) #format() strips out extra spacing
        
        # And join them together with colons
        colonized = [":".join(p) for p in clean_pairs]
        
        # So now colonized is ['"type":"stats"']
        # and we need to join those together with commas
        commas = ','.join(colonized)
        
        # And make it look like a JSON object...
        json_str = "{" + commas + "}"
        
        # And make it a python dictionary
        clean_objs.append(json.loads(json_str))
        
        #now we can cleanly get to the player stats url
        stats_url = (
            "http://goheels.com/services/responsive-roster-bio.ashx?"
            "type={type}&rp_id={rp_id}&path={path}&year={year}"
            "&player_id={player_id}"
        ).format(**clean_objs[0])
        
        return(stats_url)

In [14]:
def update_with_stats(player):
    player_url = base_url + player["href"]
    new_resp = requests.get(player_url, headers=headers)
    body_str = new_resp.content.decode('utf-8')
    parts = body_str.split('$.getJSON("/services/')[1:]
    
    # Now we run our regular expression on the joined 
    # parts (put them back in to a single string)
    # and capture the JS objects
    captured = js_obj_rx.findall(''.join(parts))
    stats_url = find_stats_url(captured)
    
    #the stats scraping begins now!
    stats_resp = requests.get(stats_url, headers=headers)
    stats_html = json.loads(stats_resp.content.decode("utf-8"))["career_stats"]
    
    #do this to avoid error if stats html is empty
    if not isinstance(stats_html, str):
        return
    html_sel = scrapy.Selector(text=stats_html)

    #scraping defensive table data
    dthead = html_sel.css('thead')[1]
    dcol_names = dthead.css('tr:nth-child(1) th')[1:].xpath('string()').extract()
    dtbody = html_sel.css('tbody')[1]
    dyears = dtbody.css('th').xpath('string()').extract()
    for o, y in enumerate(dyears):
        dcol = dtbody.css('tr')[o]
        dcol_content = dcol.css('td').xpath('string()').extract()
        for r in dcol_names:
            data={}
            for i, d in enumerate(dcol_content):
                data[dcol_names[i]] = dcol_content[i]
        dcol_title = "defensive-stats-" + str(y)
        player[dcol_title] = data
     
    #scraping offensive data table
    thead = html_sel.css('thead')[0]
    # Get all the column names from the first row's table heading (th) elements
    col_names = thead.css('tr:nth-child(1) th')[1:].xpath('string()').extract()
    tbody = html_sel.css('tbody')[0]
    years = tbody.css('th').xpath('string()').extract()
    for o, y in enumerate(years):
        col = tbody.css('tr')[o]
        col_content = col.css('td').xpath('string()').extract()
        for r in col_names:
            data={}
            for i, d in enumerate(col_names):
                data[col_names[i]] = col_content[i]
        col_title = "offensive-stats-" + str(y)
        player[col_title] = data

In [15]:
for player in players:
    update_with_bio(player)
    update_with_stats(player)
    
players

[{'#': '1',
  'Full Name': 'Mariah Evans',
  'High School': 'Lewis Palmer',
  'Hometown': 'Colorado Springs, Colo.',
  'Ht.': '5-10',
  'Pos.': 'S',
  'Previous School': '',
  'Year': 'So.',
  'defensive_stats': {'B/S': '0.02',
   'BA': '2',
   'BE': '0',
   'BHE': '27',
   'BS': '0',
   'D/S': '1.38',
   'DIG': '168',
   'PTS': '25.0',
   'PTS/S': '0.20',
   'RE': '0',
   'TB': '2'},
  'href': '/roster.aspx?rp_id=8287',
  'offensive-stats': {'%': '.478',
   'A': '503',
   'A/S': '4.12',
   'E': '1',
   'K': '12',
   'K/S': '0.10',
   'MP': '33',
   'SA': '12',
   'SA/S': '0.10',
   'SE': '26',
   'SP': '122',
   'TA': '23'}},
 {'#': '2',
  'Full Name': 'Kendra Koetter',
  'High School': 'Wesleyan School',
  'Hometown': 'Tampa, Fla.',
  'Ht.': '5-8',
  'Pos.': 'S',
  'Previous School': 'Georgia Southern',
  'Year': 'Jr.',
  'defensive_stats': {'B/S': '0.00',
   'BA': '0',
   'BE': '2',
   'BHE': '18',
   'BS': '0',
   'D/S': '1.54',
   'DIG': '146',
   'PTS': '24.0',
   'PTS/S': '0.25'

In [13]:
# with open("volleyball_data.json", "w") as volleyball_file:
#     volleyball_file.write(json.dumps(players))
to_dump = [p.copy() for p in players]
for p in to_dump:
#     p.pop('sel')
    for k in list(p.keys()):
        if 'raw' in k:
            p.pop(k)
with open('filename.json', 'w') as f:
    json.dump(to_dump, f)